#### Import necessary library

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras import layers
from keras.preprocessing import sequence
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout


In [2]:
train_dir = 'C:\\Users\\xinga\\OneDrive\\文档\\GitHub\\AI-Project\\dataset\\aclImdb\\train'
val_dir = 'C:\\Users\\xinga\\OneDrive\\文档\\GitHub\\AI-Project\\dataset\\aclImdb\\test'

batch_size = 32

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    class_names=['neg', 'pos'])

raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    val_dir,
    batch_size=batch_size,
    class_names=['neg', 'pos'])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [16]:
raw_train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [4]:
# Define the text vectorization layer
vectorize_layer = layers.TextVectorization(
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=500)

In [5]:
# Fit the text vectorization layer to the training data
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [6]:
# Define a function to apply the text vectorization layer to the data
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [7]:
# Apply the text vectorization layer to the training and validation data
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [8]:
train_ds

<MapDataset element_spec=(TensorSpec(shape=(None, 500), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [13]:
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=16))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
# model = tf.keras.Sequential([
#     layers.Embedding(input_dim=10000, output_dim=16),
#     SimpleRNN(16),
#     layers.Dense(1, activation='sigmoid')
# ])
# model = tf.keras.Sequential([
#     layers.Embedding(input_dim=10000, output_dim=64),
#     layers.LSTM(64),
#     layers.Dense(1, activation='sigmoid')
# ])


In [14]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Fit the model to the training data
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3)

Epoch 1/3
782/782 [==============================] - 433s 550ms/step - loss: 0.6933 - accuracy: 0.5002 - val_loss: 0.6931 - val_accuracy: 0.5051
Epoch 2/3
782/782 [==============================] - 441s 564ms/step - loss: 0.6940 - accuracy: 0.5082 - val_loss: 0.6935 - val_accuracy: 0.5062
Epoch 3/3
782/782 [==============================] - 442s 566ms/step - loss: 0.6892 - accuracy: 0.5098 - val_loss: 0.6953 - val_accuracy: 0.5049
